In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:
# Connect to Mongo DB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define the database and collection to hold the scraped data
db = client.mars_mission_db
collection = db.news

# Scape Source 1 - NASA Mars News

In [ ]:
# Set up Chrome Driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Define the url for NASA Mars News and use the headless browser to visit it
url = 'https://redplanetscience.com/'
browser.visit(url)

# Get the html object
html = browser.html

# Parse the html object
soup = bs(html, 'html.parser')

# Get all the relevant elements for news
results = soup.find_all('div', class_='list_text')

In [ ]:
results

In [ ]:
# Loop through results and pull title and paragraph
# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of news post
        title = result.find('div', class_='content_title').text
        # Identify and return paragraph of news post
        paragraph = result.find('div', class_='article_teaser_body').text

        # Run only if title and paragraph are available
        if (title and paragraph):
            # Print results
            print('-------------')
            print(title)
            print(paragraph)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'paragraph': paragraph,
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

# Scape Source 2 - JPL Mars Space Images - Featured Image

In [ ]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)

# Get the html object
html = browser.html

# Parse the html object
soup = bs(html, 'html.parser')

# Find a tag where class is brand_title
results = soup.find('div', class_='floating_text_area')

In [ ]:
relimageurl = results.find('a', href=True)
relimageurl['href']

In [ ]:
featured_image_url = url +relimageurl['href']
featured_image_url

# Scrape Source 3 - Mars Facts

In [10]:
# Define the source to scrap
url = 'https://galaxyfacts-mars.com/'

In [11]:
facts_list = pd.read_html(url)
facts_df = facts_list[0]
facts_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
facts_df.columns = facts_df.iloc[0]
facts_df = facts_df[1:]
facts_df

,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
facts_df = facts_df.set_index('Mars - Earth Comparison')
facts_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [17]:
facts_html = pd.to_html()
facts_html

AttributeError: module 'pandas' has no attribute 'to_html'